## Loop through all files at specified magnitude

In [2]:
# loop through all files with 20x magnitude
import cv2
import numpy as np
import math
import os 
import string 
import colorsys
from Flake_Finding_Functions import find_index, region
import sqlite3
import pandas as pd

# create class of flakes
class flakes:
    def __init__(self, image, mag, thickness, l_side, num_flakes, location):
        self.image = image
        self.mag = mag
        self.thickness = thickness
        self.l_side = l_side
        self.num_flakes = num_flakes
        self.location = location
        
mag_using = "_20x"
directory = "Images/GRAPHENE/" 
unfiltered_images = [];
#Code from: https://www.folkstalk.com/2022/10/python-loop-through-all-folders-and-subfolders-with-code-examples.html
for root, subdirectories, files in os. walk(directory):
    for subdirectory in subdirectories:
        for root2,sub2, files2 in os. walk(subdirectory):
            print()
    for file in files:
        if mag_using in os. path. join(root, file): 
            unfiltered_images.append(str(os. path. join(root, file)))

unfiltered_images = [i.replace('\\\\','////') for i in unfiltered_images]



con = sqlite3.connect('flake_characteristics'+mag_using)
c = con.cursor()
c.execute('''
          CREATE TABLE IF NOT EXISTS flakes
          (image varchar(100) PRIMARY KEY, mag int, thickness varchar(100), l_side int, location varchar(100))
          ''')
con.commit()

In [ ]:
## LOOP THROUGH IMAGES
for img in unfiltered_images:
    imageusing=img
    
    ## EDGE DETECTION ##
    edge_img = cv2.imread(imageusing)
    scale_percent = 40 # percent of original size
    width = int(edge_img.shape[1] * scale_percent / 100)
    height = int(edge_img.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized = cv2.resize(edge_img, dim, interpolation = cv2.INTER_AREA)
    resized_wo_rect=resized

    #image cleaning before finding contours (Order of Operations from article by Rouizi Yacine: 
    # https://dontrepeatyourself.org/post/edge-and-contour-detection-with-opencv-and-python/)

    #convert image to grayscale 
    edge_gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)

    #blur image
    edge_blur = cv2.GaussianBlur(edge_gray, (11,11), 1)

    # Canny Edge Detection 
    edges = cv2.Canny(image=edge_blur, threshold1=1, threshold2=10) 

    #dilate image 
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    dilate = cv2.dilate(edges, kernel, iterations=1)

    # find contours of edges
    contours, heirarchy = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
    cv2.drawContours(resized, contours, -1, (0,255,0), 1)

    #fit bounding rectangles for each contour and add them to the image
    rect_points = []
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        rect_points.append([x,y,w,h])
        cv2.rectangle(resized,(x,y),(x+w, y+h),(255,0,0),1)
        
    # Filtering of smaller noise flakes [x,y,w,h]
    # ** Change p2mm ratio for different magnitudes
    p2mm_50 = 0.45189610287498977
    p2mm_20 = 0.4854368932038835
    p2mm_2_5 = 0.876514450867052
    resized = cv2.resize(edge_img, dim, interpolation = cv2.INTER_AREA)
    good_flakes = [] # used in thickness calculations
    for rect in rect_points:
        curr_length = 0
        if rect[2] >=  rect[3]:
            curr_length = rect[2]*p2mm_20
        else: 
            curr_length = rect[3]*p2mm_20
        if curr_length > 50 : 
            # plug in proper size again 
            good_flakes.append(rect)
            #cv2.rectangle(resized,(rect[0],rect[1]),(rect[0]+rect[2], rect[1]+rect[3]),(255,0,0),1)
    
    ## THICKNESS CLASSIFYING ##
    imagehere = cv2.imread(imageusing)

    colors_count={}
    (channel_b, channel_g, channel_r) = cv2.split(imagehere)
    channel_b = channel_b.flatten()
    channel_g = channel_g.flatten()  
    channel_r = channel_r.flatten()

    for i in range(len(channel_b)):
        RGB = tuple([channel_r[i], channel_g[i], channel_b[i]])
        if RGB in colors_count:
            colors_count[RGB] += 1
        else:
            colors_count[RGB] = 1

    j=0;
    for keys in sorted(colors_count, key=colors_count.__getitem__):
        j=j+1

        if j==len(colors_count):
            max_seen=keys
            red_oi=max_seen[0]
            green_oi=max_seen[1]
            blue_oi=max_seen[2]

    #blank_img = cv2.imread("Images/Logotesting/white_sqaure.png")
    bkgnd_color = ( int (max_seen [ 2 ]), int (max_seen[ 1 ]), int (max_seen [ 0 ])) 
    #color_swatch=cv2.circle(blank_img, (200,200), 2, bkgnd_color, 150) #draw center of mass (for specific color) circle on image)               
    #cv2.imwrite(imageusing+"BACKGROUNDCOLOR.jpg", color_swatch)

    hsv_bkgnd_color=colorsys.rgb_to_hsv(bkgnd_color[0],bkgnd_color[1],bkgnd_color[2])
    hsv_bkgnd_color=list(hsv_bkgnd_color)
    hsv_bkgnd_color[0] = hsv_bkgnd_color[0]*180
    hsv_bkgnd_color[1]=hsv_bkgnd_color[1]*255

    #print("Background color in hsv: {}".format(hsv_bkgnd_color))

    h_bkgnd=hsv_bkgnd_color[0]
    s_bkgnd=hsv_bkgnd_color[1]
    v_bkgnd=hsv_bkgnd_color[2]

    i=0
    resized_blank=cv2.imread(imageusing)
    good_flakes_detected_rect=[]
    good_flakes_detected_thick=[]
    good_flakes_detected_image=[]
    thresholdpercentmask=18

    for rect in good_flakes:
        [x,y,w,h]=rect
        img_crop=resized_blank[y:y+h,x:x+w] 
        #cv2.imwrite(imageusing+'crop'+str(i)+".jpg",img_crop)
        hsv = cv2.cvtColor(img_crop, cv2.COLOR_BGR2HSV)


        #fewlayer_mask = cv2.inRange(hsv, (h_bkgnd+2, s_bkgnd+10, 0), (h_bkgnd+35, s_bkgnd+100, 255))

        fewlayer_mask = cv2.inRange(hsv, (h_bkgnd-40, s_bkgnd+10, 0), (h_bkgnd+35, s_bkgnd+100, 255))

        bilayer_mask = cv2.inRange(hsv, (h_bkgnd-35, s_bkgnd+.0001, 0), (h_bkgnd+5, s_bkgnd+20, 255))

        monolayer_mask=cv2.inRange(hsv, (h_bkgnd-31, s_bkgnd-2, 0), (h_bkgnd+4.2, s_bkgnd+20, 255)) 

        mask_array=[fewlayer_mask,bilayer_mask,monolayer_mask]

        for j,mask in enumerate(mask_array):
            print()
            if(j==0):
                print("FEW")
                flake_type_name="few"
            if(j==1):
                print("BI")
                flake_type_name="bi"
            if(j==2):
                print("MONO")
                flake_type_name="mono"
            mask_here=cv2.blur(mask, (3,3))
            mask_here=cv2.threshold(mask_here, 200, 255, cv2.THRESH_BINARY)[1]

            white_pix=np.sum(mask_here == 255)
            width = int(mask_here.shape[1])
            height = int(mask_here.shape[0])
            allpix=width*height

            ratio=white_pix/allpix *100
            #print("Flake index:{}, ratio of white to black in entire crop:{}".format(i,round(ratio,4)))

            if(ratio>thresholdpercentmask):

                M=cv2.moments(mask)
                cX=int(M["m10"]/M["m00"]) # ???
                cY=int(M["m01"]/M["m00"])
                #print("center of mass: {}, {}".format(cX,cY))

                radius=3

                X_around_COM=np.arange(cX-radius,cX+radius+1)
                Y_around_COM=np.arange(cY-radius,cY+radius+1)

                count_white_around_COM=0
                pixel_count=0

                for x_val in Y_around_COM: #and here
                    for y_val in X_around_COM:
                        #print(x_val,y_val)
                        if(x_val>=0 and x_val<height and y_val>=0 and y_val<width): #changed here
                            point=mask[x_val,y_val] ##indexing error??
                            pixel_count=pixel_count+1
                            if(point==[255]):
                                count_white_around_COM=count_white_around_COM+1


                ratio_around_COM=count_white_around_COM/pixel_count

                #print("Ratio of white around COM is {}" .format(ratio_around_COM))

                mask = cv2.circle(mask, (cX,cY), 5, bkgnd_color, 2)
                #cv2.imwrite(imageusing+'cropmaskwCOM'+str(i)+flake_type_name+'.jpg', mask)

                if(ratio_around_COM>=.9): # might want to eventally change this threshold
                    good_flakes_detected_rect.append(rect)
                    good_flakes_detected_thick.append(flake_type_name)
                    good_flakes_detected_image.append(str(imageusing))
                i=i+1

        for good_flake in good_flakes_detected_rect:
            x=find_index(good_flakes_detected_rect,good_flake)
            #print(x)

            if(len(x)>1):
                first_index=x[0]
                good_flakes_detected_rect.pop(first_index)
                good_flakes_detected_thick.pop(first_index)
                good_flakes_detected_image.pop(first_index)

        #print("Good flakes rect: {}".format(good_flakes_detected_rect))
        #print("Good flakes thickness: {}".format(good_flakes_detected_thick))
        #print("Good flakes image: {}".format(good_flakes_detected_image))
        
    ## CALCULATE LONGEST SIDE
    l_side_mm = []
    regions = []
    ## ENUMERATE
    for flake in good_flakes_detected_rect:
        w = flake[2]
        h = flake[3]
        if w > h:
            l_side = w
        else:
            l_side = h
        print(l_side)
        l_side_mm.append(l_side*p2mm_20)
    
        ## FIND LOCATION REGION
        print(region(flake[0],flake[1]))
        regions.append(region(flake[0],flake[1]))
    
    ## FILTER DATA FOR SQLite DATABASE
    img = imageusing
    mag = mag_using[1:3]
    thickness = good_flakes_detected_thick
    l_side = l_side_mm
    location = regions
    usable_flakes = []
    for i in range(len(good_flakes_detected_rect)):
        usable_flakes.append(flakes(img[i],mag[i],thickness[i],l_side[i],location[i]))
        
    ## CONNECT TO SQLite DATABASE
    ### WORK ON THIS
    table_name = 'flake_characteristics'+mag_using
    con = sqlite3.connect(table_name)
    c = con.cursor()
    
    for f in usable_flakes:
        add_to_table = "INSERT INTO flake_char_test VALUES (?, ?, ?, ?, ?, ?)"
        c.execute(add_to_table, (f.image, f.mag, f.thickness, f.l_side, f.num_flakes, f.location))
        con.commit()
    
    # printe entire table as pandas query
    #print(pd.read_sql_query("SELECT * FROM , con))
df = pd.DataFrame(c.fetchall(), columns=['product_name','price'])
print (df)

                       



FEW

BI

MONO

FEW

BI

MONO

FEW

BI

MONO

FEW

BI

MONO

FEW

BI

MONO

FEW

BI

MONO

FEW

BI

MONO

FEW

BI

MONO
